## ***Training Model***

# A Transfer Learning and Optimized CNN Based Intrusion Detection System
Data pre-processing**  
Procedures:  
&nbsp; 1): Read the dataset  
&nbsp; 2): Transform the tabular data into images  
&nbsp; 3): Display the transformed images  
&nbsp; 4): Split the training and test set  

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
#Read dataset
df=pd.read_csv('/content/drive/MyDrive/balanced_data.csv')

In [18]:
df.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,35929,2100,443,2744,49916,6,14653,65,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,1,195043,2210,52673,2608,53,17,160,91765,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,2,58449,1337,45245,1593,53,17,27,295,2,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,3,70012,696,58208,865,53,17,80,53034,4,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,4,31918,2777,443,1590,50506,6,119,71,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [19]:
df.drop(df.columns[[0]], axis=1, inplace=True)

In [20]:
Y = df.Label 
X = df.drop('Label' , axis =1)

In [21]:
X.shape

(1851553, 83)

In [22]:
Y.shape

(1851553,)

# ***PCA to reduce dimensions to 9***

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=27)
_X = pca.fit_transform(X)

In [24]:
_X.shape

(1851553, 27)

In [26]:
type(_X)

numpy.ndarray

In [27]:
finaldata = pd.DataFrame(_X)

In [28]:
finaldata.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,-3.634759e+07,-2.731521e+06,-38744.382421,-9.102812e+05,1.488298e+06,-601774.713241,-288628.199811,-177674.771568,-211877.939462,-26452.128011,...,-15119.018637,-35919.700881,12746.940366,-608.876690,23997.798092,-6645.309166,1795.652568,-51996.221431,-1956.509157,-4960.959558
1,-3.626597e+07,-2.722872e+06,-38307.402398,-1.656263e+06,1.469721e+06,-580657.527176,-260199.397541,-164061.224448,-187593.539686,-22322.031527,...,-29093.554233,-19616.181521,16254.674188,-489.082607,26914.149359,-8757.004594,16145.855547,-127267.865542,-1858.779186,-5365.627649
2,-3.634717e+07,-2.731175e+06,-38750.355066,-9.388870e+05,1.488468e+06,-601764.457588,-288570.495872,-177639.614623,-211729.480512,-26214.331486,...,-15047.018792,-35398.970130,12298.060648,-592.635065,21958.369399,-5867.770318,12307.586029,-114121.529494,-1907.699722,-4874.594843
3,-3.628116e+07,-2.716825e+06,-38397.053038,-1.660352e+06,1.468725e+06,-588455.033899,-268116.333956,-166589.736051,-186343.991470,-24062.757776,...,-18368.612475,-36435.412610,11499.630919,-585.436330,13583.069014,1012.711207,13732.791447,-125995.712993,-1683.267344,-3910.757531
4,-3.634130e+07,-2.722810e+06,-38333.914329,-1.664156e+06,1.494745e+06,-603205.453279,-289162.646044,-177922.913928,-212200.166507,-26412.311579,...,-15091.885473,-35584.211658,12458.681665,-611.992589,22608.174447,-6077.619714,8973.744947,-95326.817386,-2057.293849,-5189.166174


In [29]:
merged = pd.concat([finaldata,Y ], axis=1)

In [30]:
merged.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,Label
0,-3.634759e+07,-2.731521e+06,-38744.382421,-9.102812e+05,1.488298e+06,-601774.713241,-288628.199811,-177674.771568,-211877.939462,-26452.128011,...,-35919.700881,12746.940366,-608.876690,23997.798092,-6645.309166,1795.652568,-51996.221431,-1956.509157,-4960.959558,BENIGN
1,-3.626597e+07,-2.722872e+06,-38307.402398,-1.656263e+06,1.469721e+06,-580657.527176,-260199.397541,-164061.224448,-187593.539686,-22322.031527,...,-19616.181521,16254.674188,-489.082607,26914.149359,-8757.004594,16145.855547,-127267.865542,-1858.779186,-5365.627649,BENIGN
2,-3.634717e+07,-2.731175e+06,-38750.355066,-9.388870e+05,1.488468e+06,-601764.457588,-288570.495872,-177639.614623,-211729.480512,-26214.331486,...,-35398.970130,12298.060648,-592.635065,21958.369399,-5867.770318,12307.586029,-114121.529494,-1907.699722,-4874.594843,BENIGN
3,-3.628116e+07,-2.716825e+06,-38397.053038,-1.660352e+06,1.468725e+06,-588455.033899,-268116.333956,-166589.736051,-186343.991470,-24062.757776,...,-36435.412610,11499.630919,-585.436330,13583.069014,1012.711207,13732.791447,-125995.712993,-1683.267344,-3910.757531,BENIGN
4,-3.634130e+07,-2.722810e+06,-38333.914329,-1.664156e+06,1.494745e+06,-603205.453279,-289162.646044,-177922.913928,-212200.166507,-26412.311579,...,-35584.211658,12458.681665,-611.992589,22608.174447,-6077.619714,8973.744947,-95326.817386,-2057.293849,-5189.166174,BENIGN


In [32]:
merged.to_csv('/content/drive/MyDrive/pcadata.csv')

## Data Transformation
Convert tabular data to images
Procedures:
1. Use quantile transform to transform the original data samples into the scale of [0,255], representing pixel values

In [7]:
numeric_features = df.dtypes[df.dtypes != 'object'].index
numeric_features

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

In [8]:
scaler = QuantileTransformer() 
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [9]:
# Multiply the feature values by 255 to transform them into the scale of [0,255]
df[numeric_features] = df[numeric_features].apply(lambda x: (x*255))

In [10]:
df.describe()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,...,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06
mean,1.275287e+02,1.276880e+02,1.273960e+02,1.274476e+02,1.274950e+02,1.394323e+02,1.275085e+02,1.271347e+02,1.194408e+02,1.236704e+02,...,1.115105e+02,1.274265e+02,4.965190e+01,1.556570e+01,4.959934e+01,4.967981e+01,5.059351e+01,1.830005e+01,5.047708e+01,5.055809e+01
std,7.347422e+01,7.352517e+01,7.334071e+01,7.332814e+01,7.182030e+01,7.674451e+01,7.334319e+01,7.343971e+01,8.279394e+01,7.760842e+01,...,8.964580e+01,6.698773e+01,9.417079e+01,6.006469e+01,9.406910e+01,9.421930e+01,9.478025e+01,6.448271e+01,9.456743e+01,9.471722e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.395221e+01,6.470721e+01,6.405295e+01,5.436937e+01,4.135135e+01,8.857357e+01,6.406907e+01,6.355856e+01,0.000000e+00,7.874625e+01,...,0.000000e+00,6.075075e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.273566e+02,1.278829e+02,1.271471e+02,1.273724e+02,1.140991e+02,8.857357e+01,1.275000e+02,1.270178e+02,1.089940e+02,1.452402e+02,...,1.320946e+02,1.294144e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.909928e+02,1.905480e+02,1.907030e+02,1.763814e+02,1.747222e+02,2.550000e+02,1.906757e+02,1.905139e+02,1.902928e+02,1.941216e+02,...,1.854429e+02,1.881231e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,...,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02


All features are in the same scale of [0,255]

### Generate images for each class

In [11]:
df.Label.unique()

array(['BENIGN', 'Infiltration', 'DDoS', 'DoS slowloris',
       'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'PortScan', 'Bot', 'FTP-Patator', 'SSH-Patator'], dtype=object)

In [12]:
df0=df[df['Label']=='BENIGN'].drop(['Label'],axis=1)
df1=df[df['Label']=='Infiltration'].drop(['Label'],axis=1)
df2=df[df['Label']=='DDoS'].drop(['Label'],axis=1)
df3=df[df['Label']=='DoS slowloris'].drop(['Label'],axis=1)
df4=df[df['Label']=='DoS Slowhttptest'].drop(['Label'],axis=1)
df5=df[df['Label']=='DoS Hulk'].drop(['Label'],axis=1)
df6=df[df['Label']=='DoS GoldenEye'].drop(['Label'],axis=1)
df7=df[df['Label']=='Heartbleed'].drop(['Label'],axis=1)
df8=df[df['Label']=='PortScan'].drop(['Label'],axis=1)
df9=df[df['Label']=='Bot'].drop(['Label'],axis=1)
df10=df[df['Label']=='FTP-Patator'].drop(['Label'],axis=1)
df11=df[df['Label']=='SSH-Patator'].drop(['Label'],axis=1)

In [ ]:
# Generate 9*9 color images for class 0 (Normal)
# Change the numbers 9 to the number of features n in your dataset if you use a different dataset, reshape(n,n,3)
count=0
ims = []

image_path = "/content/drive/MyDrive/train/0/"
os.makedirs(image_path)

for i in range(0, len(df0)):  
    count=count+1
    if count<=27: 
        im=df0.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(3,3,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [15]:
len(df.iloc[0].values)

84

In [16]:
df.iloc[0].values

array([90.92183491829532, 142.4455224455224, 24.75975975975976,
       204.2042042042042, 232.01147848355973, 88.57357357357357,
       232.75184555514667, 50.92342342342342, 163.74624624624622,
       78.74624624624624, 107.97297297297297, 80.66066066066065,
       111.8018018018018, 160.55555555555557, 104.90990990990991,
       181.2312312310042, 83.21321321321321, 157.10960960960963,
       84.1066066066066, 0.0, 218.04513752814842, 227.23999077447354,
       24.504504503228226, 96.35588970524239, 42.372372372372375,
       85.63813813813813, 136.8168168168168, 123.83360633360633,
       155.9675313482953, 136.39139139139138, 129.03153153153153, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 110.14264264264264,
       65.72822822822823, 231.08847308883594, 223.09309309799403,
       162.34234234234236, 92.78528528528528, 91.38138138138137,
       106.44144144144144, 107.33483483483484, 0.0, 0.0, 0.0, 0.0, 255.0,
       0.0, 0.0, 0.0, 0.0, 91.76426426426427, 105.16516516516516,